In [1]:
import os
import pickle

import scipy
import scipy.sparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy.sparse import vstack, hstack
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Task 1

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года;
* в тестовый — турниры с dateStart из 2020 года.

In [2]:
DATA_PLAYERS_FPATH = "./data/players.pkl"
DATA_RESULTS_FPATH = "./data/results.pkl"
DATA_TOURNAMENTS_FPATH = "./data/tournaments.pkl"
TRAIN_YEAR = 2019
TEST_YEAR = 2020

In [3]:
def load_pickle(filepath: str):
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    return data

In [4]:
data_players = load_pickle(DATA_PLAYERS_FPATH)
data_results = load_pickle(DATA_RESULTS_FPATH)
data_tournaments = load_pickle(DATA_TOURNAMENTS_FPATH)

In [5]:
data_players[1]

{'id': 1, 'name': 'Алексей', 'patronymic': None, 'surname': 'Абабилов'}

In [6]:
data_results[1][0]

{'team': {'id': 242,
  'name': 'Команда Азимова',
  'town': {'id': 21, 'name': 'Баку'}},
 'mask': None,
 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
 'questionsTotal': 0,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 476,
    'name': 'Анар',
    'patronymic': 'Беюкага оглы',
    'surname': 'Азимов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 878,
    'name': 'Фариз',
    'patronymic': 'Наим оглы',
    'surname': 'Аликишибеков'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1872,
    'name': 'Аднан',
    'patronymic': 'Фариз оглы',
    'surname': 'Ахундов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 13721,
    'name': 'Балаш',
    'patronymic': 'Алекпер оглы',
    'surname': 'Касумов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'play

In [7]:
data_tournaments[1]

{'id': 1,
 'name': 'Чемпионат Южного Кавказа',
 'dateStart': '2003-07-25T00:00:00+04:00',
 'dateEnd': '2003-07-27T00:00:00+04:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/1',
 'orgcommittee': [],
 'synchData': None,
 'questionQty': None}

In [8]:
s = set()
for key in data_results.keys():
    for result in data_results[key]:
        mask = result.get("mask")
        if mask is not None:
            s.update(set(mask))
print(s)

{'X', '1', '0', '?'}


Sometimes mask includes not only "0" and "1" but also "?" and "X".

I tried to skip this masks and not include in final statistic and just change "?" to 0 and "X" to "" that works better

In [9]:
def collect_data_result(tournament_id):
    data_results_collection = []
    question_ids = []
    player_ids = []
    for result in data_results[tournament_id]:
        
        collecting_result = dict()
        mask = result.get("mask")
        team_members = result.get("teamMembers")
        # If we don't have information about team members just skip it
        if team_members is None or len(team_members) < 1:
            continue
        if mask is not None:
            mask = mask.replace("?", "0")
            mask = mask.replace("X", "")
            if mask == "":
                continue
            collecting_result["tournament_id"] = tournament_id
            collecting_result["position"] = result["position"]
            collecting_result["player_ids"] = [team_member["player"]["id"] for team_member in team_members]
            collecting_result["mask"] = list(map(int, mask))
            collecting_result["team_id"] = result["team"]["id"]
            data_results_collection.append(collecting_result)
            question_ids.extend([str(tournament_id) + "." + str(i) for i in range(len(mask))])
            player_ids.extend(collecting_result["player_ids"])
    return data_results_collection, question_ids, player_ids

In [10]:
train_data_results = []
test_data_results = []
train_question_ids = []
train_player_ids = []

train_data_tournaments = dict()
test_data_tournaments = dict()

for tournament_id in data_tournaments.keys():
    date_start = data_tournaments[tournament_id].get("dateStart")
    if date_start:
        year_start = pd.to_datetime(date_start).year
        if TRAIN_YEAR == year_start:
            data_results_collection, question_ids, player_ids = collect_data_result(tournament_id)
            train_data_results.extend(data_results_collection)
            train_data_tournaments[tournament_id] = data_tournaments[tournament_id]
            train_question_ids.extend(question_ids)
            train_player_ids.extend(player_ids)
                
        if TEST_YEAR == year_start:
            data_results_collection, _, _ = collect_data_result(tournament_id)
            test_data_results.extend(data_results_collection)
            test_data_tournaments[tournament_id] = data_tournaments[tournament_id]

train_question_ids = np.array(train_question_ids)
train_player_ids = np.unique(train_player_ids)

In [11]:
print("Train size:", len(train_data_tournaments))
print("Test size:", len(test_data_tournaments))

Train size: 687
Test size: 418


# Task 2

2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

In [12]:
train_data_tournaments[4772]

{'id': 4772,
 'name': 'Синхрон северных стран. Зимний выпуск',
 'dateStart': '2019-01-05T19:00:00+03:00',
 'dateEnd': '2019-01-09T19:00:00+03:00',
 'type': {'id': 3, 'name': 'Синхрон'},
 'season': '/seasons/52',
 'orgcommittee': [{'id': 28379,
   'name': 'Константин',
   'patronymic': 'Владимирович',
   'surname': 'Сахаров'}],
 'synchData': {'dateRequestsAllowedTo': '2019-01-09T23:59:59+03:00',
  'resultFixesTo': '2019-01-19T23:59:59+03:00',
  'resultsRecapsTo': '2019-01-11T23:59:59+03:00',
  'allowAppealCancel': True,
  'allowNarratorErrorAppeal': False,
  'dateArchivedAt': '2019-01-26T23:59:59+03:00',
  'dateDownloadQuestionsFrom': '2019-01-04T00:00:00+03:00',
  'dateDownloadQuestionsTo': '2019-01-09T19:00:00+03:00',
  'hideQuestionsTo': '2019-01-09T23:59:59+03:00',
  'hideResultsTo': '2019-01-09T23:59:59+03:00',
  'allVerdictsDone': None,
  'instantControversial': True},
 'questionQty': {'1': 12, '2': 12, '3': 12}}

In [19]:
train_data_results[0]

{'tournament_id': 4772,
 'position': 1,
 'player_ids': [6212, 18332, 18036, 22799, 15456, 26089],
 'mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0],
 'team_id': 45556}

In [20]:
players_ohe = OneHotEncoder().fit(train_player_ids.reshape(-1, 1))
questions_ohe = OneHotEncoder().fit(train_question_ids.reshape(-1, 1))

In [21]:
def get_training_data():
    x_train = []
    y_train = []

    for data_result in tqdm(train_data_results, total=len(train_data_results)):
        tournament_id = data_result["tournament_id"]
        mask = data_result["mask"]
        player_ids = data_result["player_ids"]
        question_ids = [str(tournament_id) + "." + str(i) for i in range(len(mask))]
        x_players_train = np.array([np.full((len(mask), ), _id) for _id in player_ids]).reshape(-1, 1)
        x_players_train_ohe = players_ohe.transform(x_players_train)
        x_questions_train = np.tile(question_ids, len(player_ids)).reshape(-1, 1)
        x_questions_train_ohe = questions_ohe.transform(x_questions_train)
        y_train.append(np.tile(mask, len(player_ids)).reshape(-1, 1))
        x_train.append(hstack([x_players_train_ohe, x_questions_train_ohe]))

    x_train_final = vstack(x_train)
    y_train_final = np.vstack(y_train)
    
    return x_train_final, y_train_final


def save_training_data(data, filepath, is_sparse=False):
    if is_sparse:
        scipy.sparse.save_npz(filepath, data)
    else:
        np.save(filepath, data)

        
def load_training_data(filepath, is_sparse=False):
    if is_sparse:
        return scipy.sparse.load_npz(filepath)
    return np.load(filepath)

In [22]:
# # Getting data dor training
# # It takes a lot of time to calculate (~ 1 hour), so I saved the result and will load it
# x_train, y_train = get_training_data()

  0%|          | 0/86395 [00:00<?, ?it/s]

In [24]:
# # Saving data
# # I already saved my data, so I just skipped this step

X_TRAIN_FPATH = "./data/x_train.npz"
Y_TRAIN_FPATH = "./data/y_train.npy"

# save_training_data(data=x_train, filepath=X_TRAIN_FPATH, is_sparse=True)
# save_training_data(data=y_train, filepath=Y_TRAIN_FPATH, is_sparse=False)

In [25]:
# Load saved training data
x_train = load_training_data(X_TRAIN_FPATH, is_sparse=True) 
y_train = load_training_data(Y_TRAIN_FPATH, is_sparse=False) 

In [26]:
model_log_reg = LogisticRegression(solver='saga')
model_log_reg.fit(x_train, y_train.ravel())

LogisticRegression(solver='saga')

In [27]:
df_results = pd.DataFrame(
    {
        'id': sorted(train_player_ids),
        'rating': model_log_reg.coef_[0][:len(train_player_ids)]
    }
)
df_results.sort_values("rating", ascending=False).reset_index().head()

,index,id,rating
0,3887,27403,4.101367
1,609,4270,3.974395
2,4078,28751,3.948218
3,4268,30152,3.771895
4,3960,27822,3.766546


In [28]:
top_k = 10
df_players = pd.DataFrame(data_players.values())
df_player_rating = pd.merge(df_players, df_results).sort_values("rating", ascending=False).reset_index()
df_player_rating.drop(columns="index", inplace=True)
print(f"Top {top_k} players")
df_player_rating.head(top_k)

Top 10 players


,id,name,patronymic,surname,rating
0,27403,Максим,Михайлович,Руссо,4.101367
1,4270,Александра,Владимировна,Брутер,3.974395
2,28751,Иван,Николаевич,Семушин,3.948218
3,30152,Артём,Сергеевич,Сорожкин,3.771895
4,27822,Михаил,Владимирович,Савченков,3.766546
5,30270,Сергей,Леонидович,Спешков,3.763140
6,20691,Станислав,Григорьевич,Мереминский,3.620690
7,18036,Михаил,Ильич,Левандовский,3.617614
8,26089,Ирина,Сергеевна,Прокофьева,3.572056
9,22799,Сергей,Игоревич,Николенко,3.566262


In [29]:
print(f"Last {top_k} players (according to the model)")
df_player_rating.tail(top_k)

Last 10 players (according to the model)


,id,name,patronymic,surname,rating
59091,207702,Дарья,Александровна,Безъязыкова,-3.988907
59092,207707,Антон,Денисович,Сычкин,-3.988911
59093,207704,Анастасия,Михайловна,Ильина,-3.988914
59094,207705,Кристина,Александровна,Малыгина,-3.988914
59095,207703,Дарья,Александровна,Петрушова,-3.988920
59096,209401,Михаил,Максимович,Казарин,-4.005631
59097,209400,Илья,Владиславович,Шапуров,-4.005650
59098,203842,Вероника,Андреевна,Балакина,-4.013981
59099,203845,Андрей,Кириллович,Бурштын,-4.055721
59100,203843,Анастасия,Павловна,Чурсина,-4.142071


I tried to compare my rating list with removed tournaments containing "?" and "X" and with replaced "?" to "0" and "X" to "". In the second case I got slightly better result.  
Most of the players in top 10 have good rating according to the site (almost all of them are in the top 50).  
In the end of the rating list I got players that don't have rating on the site at all.  

# Task 3

3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью,усреднённые по тестовому множеству турниров.

In [30]:
test_data_results[5]

{'tournament_id': 4957,
 'position': 5.5,
 'player_ids': [23178, 19915, 10695, 74382, 26911],
 'mask': [0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1],
 'team_id': 45367}

Let's assume that the team rating is probability of answering a question by a team. And this probability is the average probability of answering a question by its members.

In [91]:
def get_test_data():
    test_data = dict()
    medium_diff_question_idx = (
        np.abs(
            model_log_reg.coef_[0][len(train_player_ids):] - model_log_reg.coef_[0][len(train_player_ids):].mean()
        )
    ).argmin()
    medium_diff_question = questions_ohe.categories_[0][medium_diff_question_idx]

    for data_result in tqdm(test_data_results):
        true_team_rating = data_result["position"]
        team_members = []
        for player_id in data_result["player_ids"]:
            try:
                players_ohe.transform([[player_id]])
            except:
                continue
            team_members.append(player_id)
        team_members = np.array(team_members)
        if len(team_members) != 0:
            test_questions = np.full((len(team_members), 1), medium_diff_question)
            test_questions_ohe = questions_ohe.transform(test_questions)
            
            test_players_ohe = players_ohe.transform(team_members.reshape(-1, 1))
            
            x_test = hstack([test_players_ohe, test_questions_ohe])
            
            proba = model_log_reg.predict_proba(x_test)[:, 1]
            team_rating = proba.mean()
            
            team_id = data_result["team_id"]
            team_result = [team_id, true_team_rating, team_rating]
            tournament_id = data_result["tournament_id"]
            if test_data.get(tournament_id) is None:
                test_data[tournament_id] = [team_result]
            else:
                test_data[tournament_id].append(team_result)
    return test_data


def save_test_data(data, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)


def load_test_data(filepath):
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data


def get_correlations(test_data):
    spearmanr_corrs = []
    kendall_corrs = []
    for teams_result in test_data.values():
        teams_result = np.array(teams_result)
        true_team_rating, team_rating = teams_result[:, 1], teams_result[:, 2]
        spearman = scipy.stats.spearmanr(true_team_rating, team_rating).correlation
        kendall = scipy.stats.kendalltau(true_team_rating, team_rating).correlation
        if np.isfinite(spearman):
            spearmanr_corrs.append(spearman)
        if np.isfinite(kendall):
            kendall_corrs.append(kendall)
    return np.mean(spearmanr_corrs), np.mean(kendall_corrs)

In [92]:
# # Calculating test data takes also a lot of time, so I also saved the result and just load it
# test_data = get_test_data()

In [93]:
TEST_DATA_FPATH = "./data/test_data.pkl"
# save_test_data(test_data, TEST_DATA_FPATH)

In [94]:
test_data = load_test_data(TEST_DATA_FPATH)

In [96]:
spearmanr_corr, kendall_corr = get_correlations(test_data)

print(f'Корреляция Спирмена: {abs(spearmanr_corr):.4f}')
print(f'Корреляция Кендалла: {abs(kendall_corr):.4f}')

Корреляция Спирмена: 0.7851
Корреляция Кендалла: 0.6295


# Task 4

4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.